In [1]:
from src.guloso import labirinto_gulosos
from src.backtracking import labirinto_backtracking

In [2]:
maze = [
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 1]
]

In [3]:
caminho = []
visitado = [[False]*len(maze[0]) for _ in range(len(maze))]

if labirinto_gulosos(maze, 0, 0, caminho, visitado, (1,2)):
    print("Caminho encontrado (Guloso):", caminho)
else:
    print("Nenhum caminho encontrado.")


Caminho encontrado (Guloso): [(0, 0), (0, 1), (1, 1), (1, 2)]


In [4]:
caminho = []
visitado = [[False]*len(maze[0]) for _ in range(len(maze))]

if labirinto_backtracking(maze, 0, 0, caminho, visitado):
    print("Caminho encontrado (Backtracking):", caminho)
else:
    print("Nenhum caminho encontrado.")

Nenhum caminho encontrado.
